# Vectorized Operations

Vector based programming (also known as array based programming) is a programming paradigm that uses operations on arrays to execute tasks. This is in contrast to scalar based programming where operations are performed on individual elements of an array.

<center>
<img style="filter:invert(1)" width="60%" src="https://www.illustratedbytes.com/blog/series_what_is_kinetica/01-vectorize/assets/sequential.gif">  <img width="60%" style="filter:invert(1)" src="https://www.illustratedbytes.com/blog/series_what_is_kinetica/01-vectorize/assets/vectorize-2.gif">
</center>

<br/>

Vectorization operates at the level of individual instructions sent to a processor within each node. For instance, in the illustration shown here, the instruction is to add 5 to a column of numbers and copy the results to a new column B. With vectorization, all the data elements in that column are transformed simultaneously, i.e. the instruction to add 5 is applied to multiple pieces of data at the same time. This paradigm is sometimes referred to as Single Instruction Multiple Data (or SIMD).

We can think of vectorization as subdividing the work into smaller chunks that can be handled independently by different computational units at the same time.

<u>In this course, we will be minimizing the use of loops. </u>

| | | 
|:---:|:---:|
|<img src="../assets/serial2.png" alt="fishy" class="bg-primary" width="100%"  style="filter:invert(1)"> |<img src="../assets/vectorized.png" alt="fishy" class="bg-primary" width="40%"  style="filter:invert(1)"> | 
| 1. Serial / Sequential Operations | 2. Vectorized Operations | 

<br/><br/>

This is orders of magnitude faster than the conventional sequential model where each piece of data is handled one after the other in sequence.

Vectorized operations are also known as SIMD (Single Instruction Multiple Data) operations in the context of computer architecture. In contrast, scalar operations are known as SISD (Single Instruction Single Data) operations.

<center>
<img width="60%" src="../assets/simd2.png" style="filter:invert(1)">  
</center>
<br/><br/>

With vectorization, performing the same operation on a modern intel CPU is 16 times faster than the sequential mode. The performance gains on GPUs with thousands of computational cores is even greater. However, despite these remarkable performance benefits, most analytical code out there is written in the slower sequential mode. This is not a surprise, since until about a decade ago, CPU and GPU hardware could not really support vectorization for data analysis. So most implementations had to be sequential.

The last 10 years, however, have seen the rise of new technologies like CUDA from NVidia and advanced vector extensions from Intel that have dramatically shifted our ability to apply vectorization. Because of the power of vectorization, some traditional vendors now make claims about including vectorization in their offerings. But shifting to this new vectorized paradigm is not easy, since all of your code needs to be written from scratch to utilize these capabilities. 

<center>
<img width="100%" src="https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iGGVWf_CDfkE/v2/pidjEfPlU1QWZop3vfGKsrX.ke8XuWirGYh1PKgEw44kE/-1x-1.png" style="filter:invert(1)">  
</center>


<!-- 
<img src="../assets/vectorized3.png" alt="vectorized3" class="bg-primary" width="20%">  -->



Vectorization can only be applied in situations when operations at individual elements are independent of each other. For example, if we want to add two arrays, we can do so by adding each element of the first array to the corresponding element of the second array. This is a vectorized operation. 

<center><img src="../assets/vectorized_sum.png" alt="vectorized2" class="bg-primary" width="30%" style="filter:invert(1)"> </center>

However, for problems such as the Fibonacci sequence, where the value of an element depends on the values of the previous two elements, we cannot vectorize the operation. Similarly, finding minimum or maximum of an array cannot be vectorized.

Let's read in the same elections data from the previous exercise and do some data manipulation and wrangling using pandas.

In [3]:
import pandas as pd 
url = "https://raw.githubusercontent.com/fahadsultan/csc272/main/data/elections.csv"
elections = pd.read_csv(url)

## Data Alignment

`pandas` can make it much simpler to work with objects that have different indexes. For example, when you add objects, if any index pairs are not the same, the respective index in the result will be the union of the index pairs. Let’s look at an example:


In [3]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=["a", "c", "d", "e"])

s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=["a", "c", "e", "f", "g"])

s1, s2, s1 + s2

(a    7.3
 c   -2.5
 d    3.4
 e    1.5
 dtype: float64,
 a   -2.1
 c    3.6
 e   -1.5
 f    4.0
 g    3.1
 dtype: float64,
 a    5.2
 c    1.1
 d    NaN
 e    0.0
 f    NaN
 g    NaN
 dtype: float64)

The internal data alignment introduces missing values in the label locations that don’t overlap. Missing values will then propagate in further arithmetic computations.

In the case of DataFrame, alignment is performed on both rows and columns:

In [ ]:
df1 = pd.DataFrame({"A": [1, 2], "B":[3, 4]})
df2 = pd.DataFrame({"B": [5, 6], "D":[7, 8]})
df1 + df2


<br/>

## Math Operations

In native Python, we have a number of operators that we can use to manipulate data. Most, if not all, of these operators can be used with Pandas Series and DataFrames and are applied element-wise in parallel. A summary of the operators supported by Pandas is shown below:

<br/>

| Category | Operators | Supported by Pandas | Comments |
| --- | --- | --- | --- |
| Arithmetic | `+`, `-`, `*`, `/`, `%`, `//`, `**` | ✅ | Assuming comparable shapes (equal length) |
| Assignment | `=`, `+=`, `-=`, `*=`, `/=`, `%=`, `//=`, `**=` | ✅ | Assuming comparable shapes |
| Comparison | `==`, `!=`, `>`, `<`, `>=`, `<=` | ✅ | Assuming comparable shapes |
| Logical | `and`, `or`, `not` | ❌ | Use `&`, `\|`, `~` instead |
| Identity | `is`, `is not` | ✅ | Assuming comparable data type/structure |
| Membership | `in`, `not in` | ❌ | Use `isin()` method instead |
| Bitwise | `&`, `\|`, `^`, `~`, `<<`, `>>` | ❌ |  |

<br/>

The most significant difference is that <u><b>logical operators `and`, `or`, and `not` are NOT used with Pandas Series and DataFrames</u></b>. Instead, we use `&`, `|`, and `~` respectively.

Membership operators `in` and `not in` are also not used with Pandas Series and DataFrames. Instead, we use the `isin()` method.


<br/>

## Creating new columns

Creating new columns in a DataFrame is a common task when working with data. In this notebook, we will see how to create new columns in a DataFrame based on existing columns or other values.

<center><img src="https://pandas.pydata.org/docs/_images/05_newcolumn_1.svg" width="80%" style="filter:invert(1)" /></center>

New columns can be created by assigning a value to a new column name. For example, to create a new column named `new_column` with a constant value `10`, we can use the following code:


In [13]:
elections['constant'] = 10

elections.head()

,Year,Candidate,Party,Popular vote,Result,%,constant
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,10
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,10
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,10
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,10
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,10


<br/>

If we want to create a new column based on an existing column, we can refer to the existing column by its name, within the square brackets, on the right side of the assignment operator. For example, to create a new column named `new_column` with the values of the existing column `column1`, we can use the following code:

<br/>


In [14]:
elections['total_voters'] = ((elections['Popular vote']* 100) / elections['%']).astype(int)

elections.head()

,Year,Candidate,Party,Popular vote,Result,%,constant,total_voters
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,10,264413
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,10,264412
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,10,1143702
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,10,1143703
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,10,1287655


Note that the new column will have the same length as the DataFrame and the calculations are element-wise. That is, the value of the new column at row `i` will be calculated based on the value of the existing column at row `i`.

<center><img src="https://pandas.pydata.org/docs/_images/05_newcolumn_2.svg" width="80%" style="filter:invert(1)" /></center>



These element-wise operations are vectorized and are very efficient.

## Renaming columns

The [`.rename()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) method is used to rename the columns or index labels of a DataFrame.

In [ ]:
elections.rename(columns={"Candidate":"Name"})

,Year,Name,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


## Sorting Data

Ordering a `DataFrame` can be useful for isolating extreme values. For example, the first 5 entries of a row sorted in descending order (that is, from highest to lowest) are the largest 5 values. [`.sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) allows us to order a `DataFrame` or `Series` by a specified column. We can choose to either receive the rows in `ascending` order (default) or `descending` order.

In [ ]:
# Sort the "Count" column from highest to lowest
elections.sort_values(by = "%", ascending=False).head()

,Year,Candidate,Party,Popular vote,Result,%
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703
91,1936,Franklin Roosevelt,Democratic,27752648,win,60.978107
120,1972,Richard Nixon,Republican,47168710,win,60.907806
79,1920,Warren Harding,Republican,16144093,win,60.574501
133,1984,Ronald Reagan,Republican,54455472,win,59.023326


We do not need to explicitly specify the column used for sorting when calling `.value_counts()` on a `Series`. We can still specify the ordering paradigm – that is, whether values are sorted in ascending or descending order.

In [ ]:
# Sort the "Name" Series alphabetically
elections["Candidate"].sort_values(ascending=True).head()

75     Aaron S. Watkins
27      Abraham Lincoln
23      Abraham Lincoln
108     Adlai Stevenson
105     Adlai Stevenson
Name: Candidate, dtype: object